<a href="https://colab.research.google.com/github/Nandeesh-U/ZOMATO-RESTAURANT-CLUSTERING-AND-SENTIMENT-ANALYSIS/blob/main/ZOMATO_RESTAURANT_CLUSTERING_AND_SENTIMENT_ANALYSIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Problem Statement**

Zomato is an Indian restaurant aggregator and food delivery start-up founded by Deepinder Goyal and Pankaj Chaddah in 2008. Zomato provides information, menus and user-reviews of restaurants, and also has food delivery options from partner restaurants in select cities.

India is quite famous for its diverse multi cuisine available in a large number of restaurants and hotel resorts, which is reminiscent of unity in diversity. Restaurant business in India is always evolving. More Indians are warming up to the idea of eating restaurant food whether by dining outside or getting food delivered. The growing number of restaurants in every state of India has been a motivation to inspect the data to get some insights, interesting facts and figures about the Indian food industry in each city. So, this project focuses on analysing the Zomato restaurant data for each city in India.

The Project focuses on Customers and Company, you have  to analyze the sentiments of the reviews given by the customer in the data and made some useful conclusion in the form of Visualizations. Also, cluster the zomato restaurants into different segments. The data is vizualized as it becomes easy to analyse data at instant. The Analysis also solve some of the business cases that can directly help the customers finding the Best restaurant in their locality and for the company to grow up and work on the fields they are currently lagging in.

This could help in clustering the restaurants into segments. Also the data has valuable information around cuisine and costing which can be used in cost vs. benefit analysis

Data could be used for sentiment analysis. Also the metadata of reviewers can be used for identifying the critics in the industry. 

# **Attribute Information**

## **Zomato Restaurant names and Metadata**
Use this dataset for clustering part

1. Name : Name of Restaurants

2. Links : URL Links of Restaurants

3. Cost : Per person estimated Cost of dining

4. Collection : Tagging of Restaurants w.r.t. Zomato categories

5. Cuisines : Cuisines served by Restaurants

6. Timings : Restaurant Timings

## **Zomato Restaurant reviews**
Merge this dataset with Names and Matadata and then use for sentiment analysis part

1. Restaurant : Name of the Restaurant

2. Reviewer : Name of the Reviewer

3. Review : Review Text

4. Rating : Rating Provided by Reviewer

5. MetaData : Reviewer Metadata - No. of Reviews and followers

6. Time: Date and Time of Review

7. Pictures : No. of pictures posted with review

In [1]:
# Importing the libraries
from urllib.request import urlopen
import re
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
# Defining a function to scrape the content in the website and return the html script of the page
def send_request(url):
    response = requests.get(
        url='https://app.scrapingbee.com/api/v1/',
        params={
            'api_key': 'S2X6U0NGJAYG3SLEFPB80L2STD47D3Q7JC8P81J77EYXDS82UE6CBYAZP4AX9O69O0KHHY84U4QCKYTE',
            'url': url,  
        },
        
    )
    #print('Response HTTP Status Code: ', response.status_code)
    #print('Response HTTP Response Body: ', response.content)
    return response

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Reading the data into a csv
names_df = pd.read_csv('/content/drive/MyDrive/Data Squad zomato/Zomato Restaurant names and Metadata.csv')

In [5]:
names_df.head()

,Name,Links,Cost,Collections,Cuisines,Timings
0,Beyond Flavours,https://www.zomato.com/hyderabad/beyond-flavou...,800,"Food Hygiene Rated Restaurants in Hyderabad, C...","Chinese, Continental, Kebab, European, South I...","12noon to 3:30pm, 6:30pm to 11:30pm (Mon-Sun)"
1,Paradise,https://www.zomato.com/hyderabad/paradise-gach...,800,Hyderabad's Hottest,"Biryani, North Indian, Chinese",11 AM to 11 PM
2,Flechazo,https://www.zomato.com/hyderabad/flechazo-gach...,"1,300","Great Buffets, Hyderabad's Hottest","Asian, Mediterranean, North Indian, Desserts","11:30 AM to 4:30 PM, 6:30 PM to 11 PM"
3,Shah Ghouse Hotel & Restaurant,https://www.zomato.com/hyderabad/shah-ghouse-h...,800,Late Night Restaurants,"Biryani, North Indian, Chinese, Seafood, Bever...",12 Noon to 2 AM
4,Over The Moon Brew Company,https://www.zomato.com/hyderabad/over-the-moon...,"1,200","Best Bars & Pubs, Food Hygiene Rated Restauran...","Asian, Continental, North Indian, Chinese, Med...","12noon to 11pm (Mon, Tue, Wed, Thu, Sun), 12no..."


In [6]:
# Creating a new column to store the html string of each url
names_df['content'] = np.nan

In [7]:
# Scraping through each url and storing the html string in the content column of the data frame
#for i,url in enumerate(names_df['Links']):
#  response = send_request(url)
#  content = response.content
#  names_df.loc[i,'content'] = str(content)

In [8]:
# Writing the dataframe to a csv to ensure no data loss in working
#names_df.to_csv('/content/drive/MyDrive/Data Squad zomato/Nandeesh/names_df_v2')

In [9]:
# reading the dataframe from the csv file again
names_df = pd.read_csv('/content/drive/MyDrive/Data Squad zomato/Nandeesh/names_df_v2')

In [10]:
# checking for null entries
sum(names_df['content'].isnull())

0

In [11]:
names_df.head()

,Unnamed: 0,Name,Links,Cost,Collections,Cuisines,Timings,content
0,0,Beyond Flavours,https://www.zomato.com/hyderabad/beyond-flavou...,800,"Food Hygiene Rated Restaurants in Hyderabad, C...","Chinese, Continental, Kebab, European, South I...","12noon to 3:30pm, 6:30pm to 11:30pm (Mon-Sun)","b'<!DOCTYPE html><html lang=""en"" data-rh=""lang..."
1,1,Paradise,https://www.zomato.com/hyderabad/paradise-gach...,800,Hyderabad's Hottest,"Biryani, North Indian, Chinese",11 AM to 11 PM,"b'<!DOCTYPE html><html lang=""en"" data-rh=""lang..."
2,2,Flechazo,https://www.zomato.com/hyderabad/flechazo-gach...,"1,300","Great Buffets, Hyderabad's Hottest","Asian, Mediterranean, North Indian, Desserts","11:30 AM to 4:30 PM, 6:30 PM to 11 PM","b'<!DOCTYPE html><html lang=""en"" data-rh=""lang..."
3,3,Shah Ghouse Hotel & Restaurant,https://www.zomato.com/hyderabad/shah-ghouse-h...,800,Late Night Restaurants,"Biryani, North Indian, Chinese, Seafood, Bever...",12 Noon to 2 AM,"b'<!DOCTYPE html><html lang=""en"" data-rh=""lang..."
4,4,Over The Moon Brew Company,https://www.zomato.com/hyderabad/over-the-moon...,"1,200","Best Bars & Pubs, Food Hygiene Rated Restauran...","Asian, Continental, North Indian, Chinese, Med...","12noon to 11pm (Mon, Tue, Wed, Thu, Sun), 12no...","b'<!DOCTYPE html><html lang=""en"" data-rh=""lang..."


In [ ]:
# No null entries. so all the urls were scraped

In [12]:
# Picking the latitude and longitude of the restaurants location
for i, content in enumerate(names_df['content']):
  # updating the string to a soup string to easily parse
  soup = str(BeautifulSoup(names_df.loc[i,'content'],"html.parser"))

  # Parsing the latitude and longitude
  tmp = list(re.finditer('https://maps.zomato.com/',soup))
  if len(tmp) == 0:
    names_df.loc[i,'latitude'] = np.nan
    names_df.loc[i,'longitude'] = np.nan
  else:
    loc = tmp[0].span()[0]
    geo_loc = re.findall('=.+&map',soup[loc:loc+200])[0][1:-4]
    names_df.loc[i,'latitude'] = geo_loc.split(',')[0]
    names_df.loc[i,'longitude'] = geo_loc.split(',')[1]

  # Parsing the List of additional services( as a dictionary item in the dataframe column)
  tmp_loc = re.search("More Info",str(soup))
  if tmp_loc==None:
    names_df.loc[i,'additional_services'] = np.nan
  else:
    more_loc = tmp_loc.span()[0]
    tmp = soup[more_loc:]
    inds = list(re.finditer('color="#4F4F4F"',tmp))
    services = list()
    for ind in inds:
      loc = ind.span()[0]
      services.append(re.findall('>.+</p',tmp[loc:loc+50])[0][1:-3])
    names_df.loc[i,'additional_services'] = str(services)
    
  # Identifying if the restaurant has featured in any of the best lists of the city - binary variable = 1 if featured, 0 otherwise
  names_df.loc[i,'Has_Featured'] = int(len(list(re.finditer('Featured In',soup)))>0)

  # Identifying what people associate this restaurant for
  inds = list(re.finditer("People Say This Place Is Known For",str(soup)))
  if len(inds) == 0:
    names_df.loc[i,'People say this is known for'] = np.nan
  else:
    ind = inds[0].span()[0]
    tmp = soup[ind:ind+500]
    names_df.loc[i,'People say this is known for']=re.findall('color="#4F4F4F">.+</p><h3',tmp)[0][16:-7]

In [13]:
# dropping the content column
names_df.drop('content',axis = 1,inplace = True)

In [14]:
names_df.head()

,Unnamed: 0,Name,Links,Cost,Collections,Cuisines,Timings,latitude,longitude,additional_services,Has_Featured,People say this is known for
0,0,Beyond Flavours,https://www.zomato.com/hyderabad/beyond-flavou...,800,"Food Hygiene Rated Restaurants in Hyderabad, C...","Chinese, Continental, Kebab, European, South I...","12noon to 3:30pm, 6:30pm to 11:30pm (Mon-Sun)",17.4288789799,78.3739606768,"['Home Delivery', 'Takeaway Available', 'Seati...",0.0,"Ambience and Service, Courteous Staffs, Cozy, ..."
1,1,Paradise,https://www.zomato.com/hyderabad/paradise-gach...,800,Hyderabad's Hottest,"Biryani, North Indian, Chinese",11 AM to 11 PM,17.4423818301,78.3565796167,"['Home Delivery', 'Takeaway Available', 'Valet...",1.0,"Good Food Service, Good for Large Groups, Happ..."
2,2,Flechazo,https://www.zomato.com/hyderabad/flechazo-gach...,"1,300","Great Buffets, Hyderabad's Hottest","Asian, Mediterranean, North Indian, Desserts","11:30 AM to 4:30 PM, 6:30 PM to 11 PM",17.4352545759,78.3680872992,"['Home Delivery', 'Takeaway Available', 'Free ...",1.0,"Value of Money Food, Good for Large Groups, Co..."
3,3,Shah Ghouse Hotel & Restaurant,https://www.zomato.com/hyderabad/shah-ghouse-h...,800,Late Night Restaurants,"Biryani, North Indian, Chinese, Seafood, Bever...",12 Noon to 2 AM,17.4267217841,78.3764155582,"['Home Delivery', 'Takeaway Available', 'Indoo...",1.0,"Delivered on Time, Big Restaurant, Food was Go..."
4,4,Over The Moon Brew Company,https://www.zomato.com/hyderabad/over-the-moon...,"1,200","Best Bars & Pubs, Food Hygiene Rated Restauran...","Asian, Continental, North Indian, Chinese, Med...","12noon to 11pm (Mon, Tue, Wed, Thu, Sun), 12no...",17.4401549529,78.3619356528,"['Home Delivery', 'Full Bar Available', 'Free ...",1.0,"Music and Feel, Good Food and Good Service, Am..."


In [ ]:
names_df.to_csv('/content/drive/MyDrive/Data Squad zomato/Nandeesh/names_df_v3.csv')

In [16]:
names_df['People say this is known for'].values

array(['Ambience and Service, Courteous Staffs, Cozy, Dim Lighting, Beautiful Decor, Fancy Crowd',
       'Good Food Service, Good for Large Groups, Happy Place, Nice Hospitality, Ample Seating Area, Courteous Service',
       'Value of Money Food, Good for Large Groups, Comfortable Seating Area, Polite Service, Friendly Service, Excellent Experience',
       'Delivered on Time, Big Restaurant, Food was Good, Prompt Delivery, Taste is Good, Very Good Service',
       'Music and Feel, Good Food and Good Service, Ambiance and Music, Service is Great, Great Host, Serving Size',
       'Layout, Huge Place, Live Band, Lovely Ambience, Live Music, Wonderful Ambience',
       'Healthy Food, Meals, Packing, Excellent Food, Menu, Good Food</p><h3 class="sc-1sv4741-0 sc-bSbAYC dGyyGL">Average Cost</h3><p class="sc-1hez2tp-0 sc-cbMPqi cXSNdX" color="#4F4F4F">\\xe2\\x82\\xb9200 for one order (approx.)</p><p class="sc-1hez2tp-0 eTUNOO" color="#9C9C9C">Exclusive of applicable taxes and charges, if a

In [17]:
names_df['additional_services'].values

array(["['Home Delivery', 'Takeaway Available', 'Seating Available', 'Wifi', 'Buffet', 'Valet Parking Available', 'Romantic Dining', 'Indoor Seating', 'Table booking recommended']",
       "['Home Delivery', 'Takeaway Available', 'Valet Parking Available', 'Indoor Seating']",
       "['Home Delivery', 'Takeaway Available', 'Free Parking', 'Table booking recommended', 'Wifi', 'Buffet', 'Desserts and Bakes', 'Indoor Seating', 'Group Meal']",
       "['Home Delivery', 'Takeaway Available', 'Indoor Seating']",
       "['Home Delivery', 'Full Bar Available', 'Free Parking', 'Indoor Seating', 'Free Wifi', 'Smoking Area', 'Live Sports Screening', 'Live Music', 'Romantic Dining', 'Table reservation required']",
       "['Home Delivery', 'Takeaway Available', 'Full Bar Available', 'Desserts and Bakes', 'Live Sports Screening', 'Outdoor Seating', 'Serves Cocktails', 'Indoor Seating', 'Table booking recommended', 'Wifi']",
       "['Breakfast', 'Delivery Only', 'Desserts and Bakes', 'No Seating A